# Welcome to your Quantitative Social Sciences analysis toolkit!

In this notebook, you will perform the following operations:
    - load data
    - make histograms of variables of interest
    - calculate correlations between variables of interest
    - calculate group differences on variables of interest

In [1]:
import pandas as pd

df = pd.read_csv('data/anes_pilot_2018.csv', low_memory=False)

In [97]:
%matplotlib inline 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt

@interact(x=df.columns)
def categorical_plot(x='gender'):
    plt.figure(figsize=(10,5))
    ax=plt.gca()
    return df.groupby(x).size().plot(ax=ax, kind='bar')

pass

interactive(children=(Dropdown(description='x', index=284, options=('version', 'caseid', 'weight', 'weight_sps…

In [32]:
%matplotlib inline 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def cast(s):
    try:
        return float(s)
    except:
        return None

@interact(dependent=df.columns.sort_values(), independent=df.columns.sort_values())
def comparison_plot(dependent='ord_ftmetoo', independent='gender', bins=list(range(10,100,5))):
    plt.figure(figsize=(10,5))
    ax = plt.gca()
    
    print(len(df[dependent]), len(df[dependent].dropna()))

    dff = df.copy()
    dff = dff.dropna(subset=[dependent, independent])
    
    if len(dff[independent].unique()) > 15:
        plt.scatter(dff[independent], dff[dependent])
        return ax
    
    try:
        dff[dependent] = dff[dependent].apply(cast)
        if dff[dependent].mean() < 900:
            dff = dff[dff[dependent]<900]
        dff.groupby(independent)[dependent].plot(ax=ax, kind='hist', alpha=.33, bins=bins)
    except:
        dff.groupby(independent)[dependent].plot(ax=ax, kind='hist', alpha=.33, bins=bins)
        
    ax.legend()
    return ax


interactive(children=(Dropdown(description='dependent', index=428, options=('acaapprove', 'acaapprove_page_tim…

In [45]:
import statsmodels.api as sm

from statsmodels.formula.api import ols


def cast(s):
    try:
        return int(s)
    except:
        return None


@interact(dependent=df.columns.sort_values(), independent=df.columns.sort_values())
def comparison_plot(dependent='fttrump', independent='gender'):
    
    print(len(df[dependent]), len(df[dependent].dropna()))

    dff = df.copy()
    dff = dff.dropna(subset=[dependent, independent])
    
    try:
        dff[dependent] = dff[dependent].apply(cast)
        dff[independent] = dff[independent].apply(cast)
        
        if dff[dependent].mean() <900:
            dff = dff[dff[dependent]<900]
            
        results = sm.OLS(dff[dependent], dff[independent], hasconst=False).fit()
        
        if len(dff[independent].unique()) < 10:
            results = ols('{} ~ C({})'.format(dependent, independent), data=dff).fit()
        else:
            results = ols('{} ~ {}'.format(dependent, independent), data=dff).fit()
        
        print(results.summary())
        #return pd.crosstab(df[independent], df[dependent])
    
    except Exception as e:
        return str(e)


interactive(children=(Dropdown(description='dependent', index=195, options=('acaapprove', 'acaapprove_page_tim…

In [37]:
%debug

> <ipython-input-36-57ad18846727>(39)comparison_plot()
     35         print(results.summary())
     36         return pd.crosstab(df[independent], df[dependent])
     37 
     38     except Exception as e:
---> 39         return 'comparison does not run' + e.message

ipdb> u
*** Oldest frame
ipdb> e
*** NameError: name 'e' is not defined
ipdb> d
*** Newest frame
ipdb> e
*** NameError: name 'e' is not defined
ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> e
*** NameError: name 'e' is not defined
ipdb> dir()
['dependent', 'dff', 'independent', 'results']
ipdb> dependent
'fttrump'
ipdb> indpendent
*** NameError: name 'indpendent' is not defined
ipdb> results
ipdb> dir(results)
['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', 